In [1]:
import torch
import jieba
import random

import numpy as np

from data import Data
from torch import nn, optim
from datetime import datetime
from torch.utils.data import DataLoader, Dataset

d:\Program Files (x86)\Anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set random seed.
random_seed = 0

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda:0"

print(device)

cuda:0


In [4]:
classes = {
    0: "牡羊",
    1: "金牛",
    2: "雙子",
    3: "巨蠍",
    4: "獅子",
    5: "處女",
    6: "天秤",
    7: "天蠍",
    8: "射手",
    9: "魔羯",
    10: "水瓶",
    11: "雙魚",
}

In [5]:
raw = {i: open(f"./dataset/{classes[i]}.txt", "r", encoding='UTF-8').read() for i in range(12)}
data = Data(data=raw, padding_length=32)

train_raw = data.get("data")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\mamiy\AppData\Local\Temp\jieba.cache


Cleaning completed.
ToDataset completed.
Argumantation completed.


Loading model cost 0.338 seconds.
Prefix dict has been built successfully.


Tokenlization completed.
Padding completed.
Token2id completed.
Process completed.


In [6]:
len(train_raw)

5627

In [7]:
train_raw[0]

[[3411,
  5403,
  1156,
  2523,
  214,
  2792,
  50,
  1284,
  2769,
  3671,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911,
  4911],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [8]:
class CreateDataset(Dataset):
    def __init__(self, data: list, label: list):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return torch.tensor(self.data[index]), torch.tensor(self.label[index], dtype=torch.float)

    def __len__(self):
        return len(self.label)

In [9]:
d, l = [], []

for i, j in train_raw:
    d.append(i); l.append(j)

train_ds = CreateDataset(d, l)

In [10]:
train_ds[1]

(tensor([2792,   50, 1284, 2769, 3671,  214, 3411, 5403, 1156, 2523, 4911, 4911,
         4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911,
         4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911]),
 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [11]:
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)

In [12]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.linear1 = nn.Linear(256, 64)
        self.linear2 = nn.Linear(64, 1)
        self.relu = nn.ReLU(True)

    def forward(self, x: torch.Tensor):
        b = x.size(0)
        x = x.reshape(-1, 256)

        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)

        x = x.reshape(b, -1)
        x = torch.softmax(x, dim=1)

        return x.unsqueeze(2)

In [13]:
class AttentionClassifier(nn.Module):
    def __init__(self):
        super(AttentionClassifier, self).__init__()
        self.attention = Attention()
        self.linear = nn.Linear(256, 12)

    def forward(self, x: torch.Tensor):
        attention = self.attention(x)

        x = (x * attention).sum(dim=1)
        x = torch.log_softmax(self.linear(x), dim=1)

        return x, attention

In [14]:
class Model(nn.Module):
    def __init__(self) -> None:
        super(Model, self).__init__()
        self.embedding = nn.Embedding(data.get("token_len"), 64)
        self.lstm = nn.LSTM(
            input_size=64,
            hidden_size=256,
            num_layers=4,
            dropout=0.5,
            batch_first=True,
            bidirectional=True
        )
        self.linear = nn.Linear(16384, 12)
        self.classifier = AttentionClassifier()

    def forward(self, x: torch.Tensor):
        x = self.embedding(x)
        x, _ = self.lstm(x)

        x = x[:, :, :256] + x[:, :, :256]

        x, _ = self.classifier(x)

        return x

In [15]:
model = Model().to(device=device)

In [16]:
epochs = 30
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss = nn.CrossEntropyLoss()

In [17]:
# Traning history data.
train_accuracy_h = []
train_loss_h = []
validate_accuracy_h = []

In [18]:
def train(epochs: int, model: nn.Module, optimizer: optim.Optimizer, loss: nn.CrossEntropyLoss, dataloader: DataLoader):
    # Set model to training mode.
    model.train()

    for epoch in range(epochs):

        train_accuracy = 0
        train_loss = 0
        train_total = 0
        train_process = 0
        train_time = datetime.now().timestamp()

        for texts, labels in dataloader:
            texts: torch.Tensor
            labels: torch.Tensor

            texts = texts.to(device)
            labels = labels.to(device)

            outputs: torch.Tensor = model(texts)
            losses: torch.Tensor = loss(outputs, labels)

            # optimizer.zero_grad()
            for param in model.parameters(): param.grad = None

            # Backpropagation.
            losses.backward()

            # Update parameters.
            optimizer.step()

            _, predict = torch.max(outputs, 1)
            train_accuracy += sum([labels[i][predict[i]] == 1 for i in range(len(predict))])
            train_loss += losses.item()
            train_total += labels.shape[0]
            train_process += 1

            print(
                f"{datetime.now().strftime('%Y/%m/%d %H:%M:%S')} "
                f"Epoch: {epoch:03d} "
                f"Time: {datetime.now().timestamp() - train_time:.2f} "
                f"Process: {train_process / len(dataloader) * 100:.2f}% "
                f"Accuracy: {train_accuracy / train_total * 100:.2f}% "
                f"Loss: {train_loss:.3f}",
                end="\r"
            )

        train_accuracy_h.append(train_accuracy / train_total * 100)
        train_loss_h.append(train_loss)

        print()

        # Early stop.
        if train_accuracy / train_total > 0.99:
            print("Early stopped.")
            break

    # Set model to evaluation mode.
    model.eval()

In [19]:
train(epochs=epochs, model=model, optimizer=optimizer, loss=loss, dataloader=train_dl)

2024/04/02 12:35:25 Epoch: 000 Time: 9.41 Process: 100.00% Accuracy: 10.09% Loss: 1750.105
2024/04/02 12:35:35 Epoch: 001 Time: 10.13 Process: 100.00% Accuracy: 11.00% Loss: 1743.977
2024/04/02 12:35:45 Epoch: 002 Time: 10.01 Process: 100.00% Accuracy: 11.41% Loss: 1741.669
2024/04/02 12:35:55 Epoch: 003 Time: 10.49 Process: 100.00% Accuracy: 12.62% Loss: 1730.741
2024/04/02 12:36:05 Epoch: 004 Time: 10.05 Process: 100.00% Accuracy: 15.82% Loss: 1676.209
2024/04/02 12:36:15 Epoch: 005 Time: 9.92 Process: 100.00% Accuracy: 20.30% Loss: 1579.555
2024/04/02 12:36:25 Epoch: 006 Time: 9.33 Process: 100.00% Accuracy: 27.44% Loss: 1465.120
2024/04/02 12:36:34 Epoch: 007 Time: 9.13 Process: 100.00% Accuracy: 33.78% Loss: 1354.083
2024/04/02 12:36:44 Epoch: 008 Time: 9.83 Process: 100.00% Accuracy: 38.74% Loss: 1249.916
2024/04/02 12:36:53 Epoch: 009 Time: 9.27 Process: 100.00% Accuracy: 44.62% Loss: 1122.707
2024/04/02 12:37:02 Epoch: 010 Time: 9.50 Process: 100.00% Accuracy: 52.14% Loss: 971.

In [20]:
for i, j in train_dl:
    print(i)
    print(model(i.to(device)))
    print(j)
    break

tensor([[1195, 2208, 2594, 3421,   46, 1542, 3671, 4293, 4120, 4179, 1349,  589,
         4293, 4120,  820,  355, 1284, 5347, 4911, 4911, 4911, 4911, 4911, 4911,
         4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911],
        [3411, 4130, 1349, 4098, 1284, 4705, 2956, 3671,  214, 5231, 3592, 4919,
         1284, 1915, 4099,  449, 4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911,
         4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911],
        [1169,  741, 3251,  473, 4293, 1448, 3411, 1840,  191, 1284, 2133, 4293,
         5175, 5258, 3510, 4293, 2065,  883, 4460, 2176, 2270, 3671, 4293, 3411,
          479,  665, 1284, 1393, 4293, 5028,  311,  846],
        [1169, 1284, 2792, 5216, 3860, 1284, 2079, 1448, 3456, 3664, 3024, 2145,
         3331, 3003, 4895, 3671, 4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911,
         4911, 4911, 4911, 4911, 4911, 4911, 4911, 4911],
        [4089, 5090, 3998, 4548, 3543, 4293, 1762, 1284, 3338, 3998, 4404, 2113,
         3742, 2235, 4293, 3898, 3411, 

In [21]:
testset = [
    "勇敢無畏，充滿了活力和冒險精神，他們喜歡追求挑戰，敢於冒險，常常是行動派的領導者。",
    "穩重可靠，以堅韌的意志力和耐心著稱，他們注重安全和舒適，並對物質生活有著強烈的執著。",
    "機智聰明，好奇心旺盛，喜歡交際和表達自己，具有多才多藝的特質，常常充滿了靈活的思維和活力。",
    "情感豐富，善解人意，對家庭和親密關係非常重視，他們總是充滿了溫柔和關懷，是很好的傾聽者和支持者。",
    "自信大方，追求著成為焦點的慾望，他們充滿了熱情和活力，喜歡引領和影響身邊的人，時常展現出優越感和領導能力。",
    "細心謹慎，追求完美，他們善於分析和解決問題，注重細節和有組織性，常常是值得信賴的夥伴和顧問。",
    "追求和諧，優雅而公正，他們注重平衡和公平，善於溝通協調，是很好的調解者和中介者。",
    "神秘內斂，充滿了熱情和直覺，他們擁有強烈的意志力和洞察力，常常是充滿挑戰性和魅力的個體。",
    "自由奔放，熱愛冒險和探索，他們追求著廣闊的視野和新鮮的體驗，時常充滿了樂觀和幽默。",
    "勤奮負責，追求事業成功和社會地位，他們具有堅毅的意志力和耐心，常常是穩健和實際的決策者。",
    "獨立思考，充滿了理想主義和創意，他們追求著獨特的生活方式和社會價值觀，常常是前衛和不拘一格的個體。",
    "敏感善良，充滿了同情心和想像力，他們常常是理想主義者和夢想家，追求著內心的情感和精神實踐。",
    "勇敢無畏，充滿活力，喜歡追求挑戰，常常是行動派的領導者。",
    "穩重可靠，堅韌耐心，注重安全舒適，對物質生活有強烈執著。",
    "機智聰明，好奇心旺盛，善於交際表達，充滿靈活思維和活力。",
    "情感豐富，善解人意，重視家庭和親密關係，溫柔關懷，傾聽支持者。",
    "自信大方，追求成為焦點，充滿熱情活力，喜歡引領影響身邊的人。",
    "細心謹慎，追求完美，善於分析解決問題，注重細節有組織性。",
    "追求和諧，優雅公正，注重平衡公平，善於溝通協調。",
    "神秘內斂，熱情直覺，意志力洞察力強，充滿挑戰性和魅力。",
    "自由奔放，熱愛冒險探索，追求廣闊視野和新鮮體驗，樂觀幽默。",
    "勤奮負責，追求事業成功社會地位，具堅毅意志力和耐心，穩健決策者。",
    "獨立思考，理想主義創意，追求獨特生活方式和價值觀，前衛不拘一格。",
    "敏感善良，同情心想像力豐富，理想主義夢想家，追求內心情感精神實踐。",
    "勇敢果敢，充滿活力，愛冒險。",
    "穩重堅定，堅持自我價值觀。",
    "靈活機智，善於溝通交際。",
    "情感豐富，家庭意識強。",
    "自信領導，熱情奔放。",
    "細心謹慎，追求完美。",
    "追求和諧，公正公平。",
    "神秘敏感，探索深度。",
    "自由探險，樂觀向上。",
    "勤奮穩健，追求成功。",
    "獨立創新，理想主義者。",
    "敏感浪漫，夢想家。",
    "生命充滿了勇氣與活力，他們勇於面對挑戰，永不退縮，常常是行動派的領袖，樂於率先嘗試新事物。",
    "展現出穩重且堅定的品性，他們始終堅持自身的價值觀與信念，喜歡在自己熟悉的領域中深耕不輟。",
    "擁有機智敏捷的頭腦，他們善於溝通交際，充滿活力與靈活的思維，對於各種新奇的事物都抱有濃厚的好奇心。",
    "的人情感豐富且懂得關懷，他們對家庭有著極為強烈的連結感，願意為了家人無條件地付出與奉獻。",
    "展現出極度的自信與熱情，他們常常是眾人注目的焦點，充滿著領導力與活力，喜歡成為團體中的中心人物。",
    "的人細心謹慎，追求完美與規律，他們擁有出色的分析能力與解決問題的技巧，喜歡保持事情的有序與井然有序。",
    "追求和諧與公正，他們擅長於溝通協調，注重平衡與公平，善於解決衝突，是團隊中不可或缺的調解者。",
    "的人神秘內斂，充滿了深度與直覺，他們擁有強烈的意志力與洞察力，總是對事情有著深入的探索與研究。",
    "熱愛自由與探險，他們樂觀向上，喜歡挑戰與冒險，追求著廣闊的視野與新鮮的體驗，不斷探索未知的領域。",
    "的人勤奮穩健，追求事業成功與社會地位，他們具有堅毅的意志力與耐心，是穩健可靠的決策者與領導者。",
    "獨立創新，理想主義者，他們勇於打破傳統的束縛，追求獨特的生活方式與社會價值觀，常常是前衛與不拘一格的個體。",
    "的人敏感浪漫，是真正的夢想家，他們充滿了同情心與想像力，常常是理想主義者，追求內心情感與精神的實踐。",
]

answare = [
    '牡羊', '金牛', '雙子', '巨蠍', '獅子', '處女', '天秤', '天蠍', '射手', '魔羯', '水瓶', '雙魚',
    '牡羊', '金牛', '雙子', '巨蠍', '獅子', '處女', '天秤', '天蠍', '射手', '魔羯', '水瓶', '雙魚',
    '牡羊', '金牛', '雙子', '巨蠍', '獅子', '處女', '天秤', '天蠍', '射手', '魔羯', '水瓶', '雙魚',
    '牡羊', '金牛', '雙子', '巨蠍', '獅子', '處女', '天秤', '天蠍', '射手', '魔羯', '水瓶', '雙魚',
]

In [22]:
def process(testset: list) -> torch.Tensor:
    result = []

    for line in testset:
        temp = jieba.lcut(line)
        temp = temp + ["<PAD>"] * (32 - len(temp))
        temp = [data.w2i[x] if x in data.w2i else data.w2i["<PAD>"] for x in temp][:32]
        result.append(temp)

    return result

In [23]:
test = process(testset)

In [24]:
model.eval()

Model(
  (embedding): Embedding(5470, 64)
  (lstm): LSTM(64, 256, num_layers=4, batch_first=True, dropout=0.5, bidirectional=True)
  (linear): Linear(in_features=16384, out_features=12, bias=True)
  (classifier): AttentionClassifier(
    (attention): Attention(
      (linear1): Linear(in_features=256, out_features=64, bias=True)
      (linear2): Linear(in_features=64, out_features=1, bias=True)
      (relu): ReLU(inplace=True)
    )
    (linear): Linear(in_features=256, out_features=12, bias=True)
  )
)

In [25]:
result = model(torch.tensor(test).to(device))
print(result)

tensor([[-1.8120e-05, -1.9818e+01, -1.8563e+01, -2.4603e+01, -1.3343e+01,
         -2.0963e+01, -2.3896e+01, -1.6780e+01, -1.1008e+01, -2.8327e+01,
         -2.7027e+01, -2.3756e+01],
        [-1.1979e+01, -8.3695e-01, -6.3657e+00, -2.4787e+00, -1.0015e+01,
         -7.5398e+00, -3.8698e+00, -7.9994e-01, -1.1511e+01, -6.8414e+00,
         -6.5932e+00, -4.8101e+00],
        [-1.0268e+01, -4.8108e+00, -2.4234e-01, -1.3628e+01, -3.3417e+00,
         -8.1393e+00, -2.5979e+00, -9.1532e+00, -3.7665e+00, -1.8564e+01,
         -1.3129e+01, -2.6078e+00],
        [-1.8752e+01, -5.1291e+00, -4.6999e-01, -1.1184e+00, -1.1203e+01,
         -1.1297e+01, -3.1730e+00, -9.8429e+00, -8.6751e+00, -1.7246e+01,
         -1.2159e+01, -8.8168e+00],
        [-2.9810e+00, -1.7192e+01, -1.0166e+01, -1.4512e+01, -9.2100e-01,
         -1.3714e+01, -1.5234e+01, -5.9668e-01, -8.2355e+00, -2.1062e+01,
         -1.7055e+01, -8.5395e+00],
        [-1.8949e+01, -8.7164e+00, -8.2307e+00, -2.0350e+01, -1.3415e+01,
      

In [26]:
result = result.tolist()

In [27]:
# Normolization
results = []

for i in range(len(result)):
    temp = result[i]
    temp = [x - min(temp) for x in temp]
    temp = [x / max(temp) for x in temp]
    temp = [round(x, 3) for x in temp]

    results.append(temp)

In [28]:
count = 0

for i, r in enumerate(results):
    t = classes[int(torch.argmax(torch.tensor(r)))]
    if t == answare[i]:
        count += 1
        print(t + "✅")
    else:
        print(t + " <- " + answare[i])

print(f"Correct: {count}/{len(answare)} ({(count / len(answare)) * 100 :.2f}%)")

牡羊✅
天蠍 <- 金牛
雙子✅
雙子 <- 巨蠍
天蠍 <- 獅子
處女✅
天秤✅
射手 <- 天蠍
射手✅
天蠍 <- 魔羯
射手 <- 水瓶
雙魚✅
牡羊✅
天蠍 <- 金牛
雙魚 <- 雙子
巨蠍✅
獅子✅
處女✅
天秤✅
天蠍✅
射手✅
天蠍 <- 魔羯
水瓶✅
巨蠍 <- 雙魚
獅子 <- 牡羊
金牛✅
雙子✅
巨蠍✅
牡羊 <- 獅子
處女✅
天秤✅
天蠍✅
射手✅
魔羯✅
水瓶✅
雙魚✅
射手 <- 牡羊
天蠍 <- 金牛
雙子✅
巨蠍✅
獅子✅
雙魚 <- 處女
天秤✅
天蠍✅
射手✅
金牛 <- 魔羯
水瓶✅
魔羯 <- 雙魚
Correct: 31/48 (64.58%)


In [29]:
import matplotlib.pyplot as plt
import numpy as np

In [30]:
# angles_A = np.linspace(start=0, stop=2*np.pi, num=len(result)+1, endpoint=True)
# values_A = np.concatenate((result, [result[0]]))

# fig, ax = plt.subplots(1, 1, figsize=(5, 5), subplot_kw={'projection': 'polar'})
# ax.plot(angles_A, values_A, 'o-', color="blue", label="A")

# ax.fill(angles_A, values_A, alpha=0.3, color="blue")
# ax.set_thetagrids(angles_A[:-1] * 180 / np.pi, range(12), fontsize=15)
# ax.set_theta_zero_location('N')

In [31]:
# Please check the virtual input of the ONNX model.
torch.onnx.export(model, torch.tensor([test[0]]).to(device=device), 'constellator.onnx')

d:\Program Files (x86)\Anaconda\envs\pytorch\lib\site-packages\torch\onnx\symbolic_opset9.py:4315: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
d:\Program Files (x86)\Anaconda\envs\pytorch\lib\site-packages\torch\onnx\_internal\jit_utils.py:258: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\jit\passes\onnx\shape_type_inference.cpp:1888.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
d:\Program Files (x86)\Anaconda\envs\pytorch\lib\site-packages\torch\onnx\utils.py:687: UserWarning: The shap

In [32]:
import onnx
import onnx.helper

onnx_model = onnx.load('./constellator.onnx')
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch_jit (
  %input.1[INT64, 1x32]
) initializers (
  %embedding.weight[FLOAT, 5470x64]
  %classifier.attention.linear1.weight[FLOAT, 64x256]
  %classifier.attention.linear1.bias[FLOAT, 64]
  %classifier.attention.linear2.weight[FLOAT, 1x64]
  %classifier.attention.linear2.bias[FLOAT, 1]
  %classifier.linear.weight[FLOAT, 12x256]
  %classifier.linear.bias[FLOAT, 12]
  %onnx::LSTM_742[FLOAT, 2x2048]
  %onnx::LSTM_743[FLOAT, 2x1024x64]
  %onnx::LSTM_744[FLOAT, 2x1024x256]
  %onnx::LSTM_789[FLOAT, 2x2048]
  %onnx::LSTM_790[FLOAT, 2x1024x512]
  %onnx::LSTM_791[FLOAT, 2x1024x256]
  %onnx::LSTM_836[FLOAT, 2x2048]
  %onnx::LSTM_837[FLOAT, 2x1024x512]
  %onnx::LSTM_838[FLOAT, 2x1024x256]
  %onnx::LSTM_883[FLOAT, 2x2048]
  %onnx::LSTM_884[FLOAT, 2x1024x512]
  %onnx::LSTM_885[FLOAT, 2x1024x256]
) {
  %/embedding/Gather_output_0 = Gather(%embedding.weight, %input.1)
  %/lstm/Transpose_output_0 = Transpose[perm = [1, 0, 2]](%/embedding/Gather_output_0)
  %/lstm/Constant_output_0 = Constant[

In [35]:
import os
print(f"ONNX model size: {os.path.getsize('./constellator.onnx') / 1048576:.2f} MB")

ONNX model size: 22.00 MB
